Code

https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment

Video

https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbk1Iang2ME0wODgzTlctRVl6LUdBRFU0WFloZ3xBQ3Jtc0tua2k5cFpNbzlNX2UxWmlzXzl5MzlZX3VPd0RoZ1Y3UVpEUjl5MWkweWs4TFM2MmpYWUdESEI0VmU2TndCVkJUaWFUUmZraVFGN3p5WWdyalRzOFhxcmdfU2tRSkwzdGlyMDN3eGFVRVh5RWhQWklicw&q=https%3A%2F%2Fgithub.com%2Fnicknochnack%2FOpenAI-Reinforcement-Learning-with-Custom-Environment&v=bD6V3rcr_54

Solution to compatibility problems

https://stackoverflow.com/questions/59765784/attributeerror-sequential-object-has-no-attribute-get-distribution-strategy


# 0. Install Dependencies

In [ ]:
# !pip install tensorflow
#!pip install gym
#!pip install keras
# !pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [73]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [76]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [77]:
env = ShowerEnv()

In [78]:
env.observation_space.sample()

array([30.645672], dtype=float32)

In [79]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render(mode="ipython")
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-46
Episode:2 Score:22
Episode:3 Score:-10
Episode:4 Score:-48
Episode:5 Score:-32
Episode:6 Score:-44
Episode:7 Score:8
Episode:8 Score:-32
Episode:9 Score:-50
Episode:10 Score:-6


# 2. Create a Deep Learning Model with Keras

In [80]:
import tensorflow.keras as keras

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import adam_v2

In [81]:
states = env.observation_space.shape
actions = env.action_space.n

In [82]:
states

(1,)

In [83]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [84]:
# del model 

In [85]:
model = build_model(states, actions)

In [86]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 24)                48        
_________________________________________________________________
dense_23 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [87]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [88]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [89]:
dqn = build_agent(model, actions)

In [90]:
dqn.compile(adam_v2.Adam(learning_rate=1e-3), metrics=['mae'])

In [91]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 112s 11ms/step - reward: -0.6304
166 episodes - episode_reward: -37.855 [-60.000, 32.000] - loss: 1.326 - mae: 9.147 - mean_q: -10.958

Interval 2 (10000 steps performed)
 9343/10000 [===========================>..] - ETA: 7s - reward: -0.5558done, took 215.735 seconds


In [92]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 58.000, steps: 60
Episode 5: reward: 58.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 58.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 58.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 58.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 58.000, steps: 60
Episode 22: reward: 58.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st

In [93]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 58.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 58.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60


# 4. Reloading Agent from Memory

In [94]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [95]:
del model
del dqn
del env

In [96]:
# env = gym.make('CartPole-v0')
env = ShowerEnv()
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(adam_v2.Adam(learning_rate=1e-3), metrics=['mae'])

In [97]:
dqn.load_weights('dqn_weights.h5f')

In [98]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: 58.000, steps: 60
Episode 2: reward: 58.000, steps: 60
Episode 3: reward: 58.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 58.000, steps: 60
